네이버 쇼핑 검색결과 페이지를 크롤링해 본다. 기본 크롤링을 통해 이미지 주소, 상품명, 상세페이지 주소, 광고여부, 최저가, 카테고리 목록, 상품스펙 내역, 리뷰수, 별점을 크롤링하고, 찜 갯수는 Ajax통신을 통해 처리되기에 이를 추가 처리해 본다.

최종 정리된 데이터는 Pandas를 통해서 정리하고, 엑셀로 저장하기까지 실습해 본다.

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
from datetime import datetime

In [2]:
page_url = "https://search.shopping.naver.com/search/all.nhn"
params = "query=%EC%97%90%EC%96%B4%ED%8C%9F&cat_id=&frm=NVSHATC"
params = {
    'query': '에어팟'
}
res = requests.get(page_url, params=params)
res

<Response [200]>

In [3]:
html = res.text
soup = BeautifulSoup(html, 'html.parser')
tag_list = soup.select('._itemSection')
len(tag_list)

40

In [4]:
tag = tag_list[0]

In [5]:
title_tag = tag.select_one('.tit a')
goods_title = title_tag['title']
goods_url = title_tag['href']
goods_title, goods_url

('애플 에어팟 2세대 유선충전 (MV7N2KH/A)',
 'http://cr2.shopping.naver.com/adcr.nhn?x=dmlt%2BeC%2FIZnLSYqCuDiEw%2F%2F%2F%2Fw%3D%3Dsy252%2F5APCDI6sPMLqdl%2FzeE%2ByyAmQ9x%2FR0kyMqZatvhFxXq5RRrAbf2VfWG%2F%2B5KyG3FTuSabimQcXadx6gMDifJozvsg3RgcBeele%2BgCodJpOMV%2FVMpBMPerVw3rrKOj9HfrToLB%2B7smwXFR6IHuaIYlFAdf1OFsetsPTCIBnAs9DjdP86FOmBYD8KQqXSY%2FncdTCtt%2F7lJTxN9Crar7aGVgIqH44kupCBw98666579Bujm4MX42wGoUmDg%2FfMJZn4uC5Vw0Cyqfuwfx9OHQ1fiEVUaghBiTRZLmcSQRXUzUeXa8sMOm%2Fw%2FIwVfkZNoMzOQHxQcnl1ni2z%2FuQqAxpGM5ywTNbvkDUq403b3HrmOv1Qo7fM3J%2Fe%2FVhnj5JjQePvR5S3MHMC%2B5X9IQfTqkGmR2%2F2OpnSCA1oFvopKoEUgdYg%2BpNYu6zVojmz%2B0MnslWlc3GU8aKcnUL4GPvJkMtNhmPfoHtyjK5o6ZRJDo3Ezm3JcSERcasXm6F5Jr3gRnL%2FzNN8WO%2FBz8oyfJ9hHa5Psu%2BATfiGvqUR3U7Wnmc6mBeJ4godZtaVH0MWB1JduemjM8uPl4CGWehrQz9anenDlF4yRaLiecz2rtHRT%2F4YvcNv7YKvSU%2B6FVFiNXvW172xdJ2VAgqqT5wTXJHNGmCfV%2Bfr2ImTWatmjM4eBxUXrL8owKMh5kzzgOxohVjlPcj5f2GL1%2BHjullhYbgFWygQ%3D%3D')

In [6]:
goods_img_url = tag.select_one('._productLazyImg')['data-original']
goods_price = int(tag.select_one('._price_reload').text.replace(',', ''))

In [7]:
goods_cat = [tag.text for tag in tag.select('.depth a')]
goods_cat

['디지털/가전', '음향가전', '블루투스셋', '블루투스이어폰/이어셋']

In [8]:
spec = [tag.text for tag in tag.select('.detail a')]

spec_list = []

for s in spec:
    try:
        key, value = s.split(" : ", 1)
        spec_list.append([key.strip(), value.strip()])
    except ValueError:
        key, value = spec_list[-1]  # 제일 마지막 스펙 리스트 가져오기
        if not isinstance(value, list):  # 마지막 스펙의 value 포맷이 list가 아니라면,
            value = [value]  # value 포맷을 리스트화 시키고,
        value.append(s.strip()) # value 리스트에 s를 추가시킨다.
        spec_list[-1] = key, value  # 그리고 제일 마지막 리스트에 key, value를 재입력한다.

spec = dict(spec_list)

In [9]:
review_count = int(tag.select('.etc em')[1].text.replace(',', ''))

def get_star_point(tag):
    if tag.select_one('.star_graph span')['style']:  # star_point가 있는지 체크
        star_string = tag.select_one('.star_graph span')['style']
        matched = re.match(r'width:([\d\.]+)%', star_string)
        if matched:
            star_point = float(matched.group(1)) * 5 / 100  # 96.5 : 100 = x : 5
        else:
            star_point = None
    else:
        start_point = None
    return star_point

get_star_point(tag)

4.8

In [10]:
created_at_string = tag.select_one('.date').text.replace('등록일', "").strip()
jjim_tag = tag.select_one('._jjim em')
nvmid = jjim_tag['nvmid']

In [11]:
is_ad = "ad" in tag['class']

### 정보 합치기

In [12]:
def get_spec(tag):
    spec = [tag.text for tag in tag.select('.detail a')]

    spec_list = []

    for s in spec:
        try:
            key, value = s.split(" : ", 1)
            spec_list.append([key.strip(), value.strip()])
        except ValueError:
            key, value = spec_list[-1]  # 제일 마지막 스펙 리스트 가져오기
            if not isinstance(value, list):  # 마지막 스펙의 value 포맷이 list가 아니라면,
                value = [value]  # value 포맷을 리스트화 시키고,
            value.append(s.strip()) # value 리스트에 s를 추가시킨다.
            spec_list[-1] = key, value  # 그리고 제일 마지막 리스트에 key, value를 재입력한다.
            
    return dict(spec_list)

def get_star_point(tag):
    if tag.select_one('.etc .star_graph') == True:  # star_point가 있는지 체크
        star_string = tag.select_one('.etc .star_graph span')['style']
        matched = re.match(r'width:([\d\.]+)%', star_string)
        if matched:
            return float(matched.group(1)) * 5 / 100  # 96.5 : 100 = x : 5
        else:
            return None
    else:
        return None

def get_created_date(tag):
    if tag.select('.etc .date'):
        return tag.select_one('.etc .date').text.replace('등록일', "").strip()
    else:
        return None
    
def get_nvmid(tag):
    if tag.select_one('._jjim'):
        return tag.select_one('._jjim')["data-nv-mid"]
    else:
        return tag.select('.etc a')[2]['data-nv-mid']

def get_review_count(tag):
    review_count = tag.select('.etc em')[1].text
    if review_count:
        return int(review_count.replace(",", ""))
    else:
        return "empty"   
    
def get_chipid(tag):
    try:
        return tag["data-mall-pid"]
    except:
        return ""


goods_list = []

for tag in tag_list:
    title_tag = tag.select_one('.tit a')
    
    goods_list.append(dict(
        goods_title = title_tag.text, 
        goods_url = title_tag['href'],
        goods_img_url = tag.select_one('._productLazyImg')['data-original'],
        goods_price = int(tag.select_one('.num').text.replace(',', '')),
        goods_cat = [tag.text for tag in tag.select('.depth a')],
        spec = get_spec(tag),
        review_count = get_review_count(tag),
        star_point = get_star_point(tag),
        created_at = get_created_date(tag),
        nvmid = get_nvmid(tag), 
        is_ad = "ad" in tag['class'],
        chpid = get_chipid(tag)
    ))

goods_list

[{'goods_title': '애플 에어팟 2세대 유선충전 (MV7N2KH/A)',
  'goods_url': 'http://cr2.shopping.naver.com/adcr.nhn?x=dmlt%2BeC%2FIZnLSYqCuDiEw%2F%2F%2F%2Fw%3D%3Dsy252%2F5APCDI6sPMLqdl%2FzeE%2ByyAmQ9x%2FR0kyMqZatvhFxXq5RRrAbf2VfWG%2F%2B5KyG3FTuSabimQcXadx6gMDifJozvsg3RgcBeele%2BgCodJpOMV%2FVMpBMPerVw3rrKOj9HfrToLB%2B7smwXFR6IHuaIYlFAdf1OFsetsPTCIBnAs9DjdP86FOmBYD8KQqXSY%2FncdTCtt%2F7lJTxN9Crar7aGVgIqH44kupCBw98666579Bujm4MX42wGoUmDg%2FfMJZn4uC5Vw0Cyqfuwfx9OHQ1fiEVUaghBiTRZLmcSQRXUzUeXa8sMOm%2Fw%2FIwVfkZNoMzOQHxQcnl1ni2z%2FuQqAxpGM5ywTNbvkDUq403b3HrmOv1Qo7fM3J%2Fe%2FVhnj5JjQePvR5S3MHMC%2B5X9IQfTqkGmR2%2F2OpnSCA1oFvopKoEUgdYg%2BpNYu6zVojmz%2B0MnslWlc3GU8aKcnUL4GPvJkMtNhmPfoHtyjK5o6ZRJDo3Ezm3JcSERcasXm6F5Jr3gRnL%2FzNN8WO%2FBz8oyfJ9hHa5Psu%2BATfiGvqUR3U7Wnmc6mBeJ4godZtaVH0MWB1JduemjM8uPl4CGWehrQz9anenDlF4yRaLiecz2rtHRT%2F4YvcNv7YKvSU%2B6FVFiNXvW172xdJ2VAgqqT5wTXJHNGmCfV%2Bfr2ImTWatmjM4eBxUXrL8owKMh5kzzgOxohVjlPcj5f2GL1%2BHjullhYbgFWygQ%3D%3D',
  'goods_img_url': 'https://shopping-phinf.pstatic.net/main

In [13]:
df = pd.DataFrame(goods_list)
df.shape

(40, 12)

In [14]:
df.head()

,goods_title,goods_url,goods_img_url,goods_price,goods_cat,spec,review_count,star_point,created_at,nvmid,is_ad,chpid
0,애플 에어팟 2세대 유선충전 (MV7N2KH/A),http://cr2.shopping.naver.com/adcr.nhn?x=dmlt%...,https://shopping-phinf.pstatic.net/main_186220...,89400,"[디지털/가전, 음향가전, 블루투스셋, 블루투스이어폰/이어셋]","{'종류': '이어셋', '무선방식': '블루투스(애플H1칩탑재)', '통화': '...",11192,None,2019.04.,18622086330,False,
1,애플 에어팟 1세대 (MMEF2KH/A),http://cr2.shopping.naver.com/adcr.nhn?x=Et7Rn...,https://shopping-phinf.pstatic.net/main_107769...,144900,"[디지털/가전, 음향가전, 블루투스셋, 블루투스이어폰/이어셋]","{'채널': '스테레오', '무선방식': '블루투스(애플W1칩탑재)', '구성품':...",11603,None,2016.12.,10776906666,False,
2,애플 에어팟 2세대 무선충전 (MRXJ2KH/A),http://cr2.shopping.naver.com/adcr.nhn?x=OIa32...,https://shopping-phinf.pstatic.net/main_187548...,82900,"[디지털/가전, 음향가전, 블루투스셋, 블루투스이어폰/이어셋]","{'무선방식': '블루투스(애플H1칩탑재)', '통화': '3시간', '구성품': ...",3652,None,2019.04.,18754856889,False,
3,애플 에어팟 왼쪽 오른쪽 단품 한쪽 1세대 2세대 당일발송 국내정식발매품 에어팟2 2세대,http://cr2.shopping.naver.com/adcr.nhn?x=0gJJB...,https://shopping-phinf.pstatic.net/main_819460...,67000,"[디지털/가전, 음향가전, 블루투스셋, 블루투스이어폰/이어셋]","{'종류': '이어셋', '채널': '스테레오', '무선방식': '블루투스(애플W1...",796,None,2019.03.,81946053258,False,4401530808
4,[MLC] 애플 에어팟 2세대 유선/무선충전,http://cr2.shopping.naver.com/adcrNoti.nhn?x=f...,https://shopping-phinf.pstatic.net/main_197390...,179450,"[디지털/가전, 음향가전, 블루투스셋, 블루투스이어폰/이어셋]",{},497,None,2019.06.,19739090251,False,57360784


In [15]:
','.join(df.nvmid)

'18622086330,10776906666,18754856889,81946053258,19739090251,82003872939,19562859308,20991114077,18344345265,82180951465,20912358642,21086494436,82172274887,19362931639,18483728924,82094491384,21005126553,20952242897,82170177844,20987723498,19497648827,20833009935,82160944962,18735689675,18428627338,19735390207,81952924531,20469790136,21085994495,82214473403,19699444396,20190400738,82193831022,19795740731,15142858596,20775209086,20857207385,21027100371,19605906124,21099236354'

In [16]:
# 찜정보 가져오기
url = "https://search.shopping.naver.com/search/ajax/keep.nhn"

data = {
    'nvMid': ','.join(df.nvmid),
    'chpid': ','.join(df.chpid),    
}
# POST 이기 때문에 Form Data 정보 필요

res_jjim = requests.post(url, data=data)
res_jjim

<Response [200]>

In [17]:
jjim_count_list = res_jjim.json()['keepCount']
len(jjim_count_list)

40

In [18]:
df["keep_count"] = jjim_count_list

In [19]:
df[['goods_title', 'nvmid', 'keep_count']]

,goods_title,nvmid,keep_count
0,애플 에어팟 2세대 유선충전 (MV7N2KH/A),18622086330,4487
1,애플 에어팟 1세대 (MMEF2KH/A),10776906666,3166
2,애플 에어팟 2세대 무선충전 (MRXJ2KH/A),18754856889,2579
3,애플 에어팟 왼쪽 오른쪽 단품 한쪽 1세대 2세대 당일발송 국내정식발매품 에어팟2 2세대,81946053258,643
4,[MLC] 애플 에어팟 2세대 유선/무선충전,19739090251,0
5,국내정품 에어팟2 세대 AirPods2 오늘출발 내일도착 우체국 당일배송 리뷰이벤트진행,82003872939,3106
6,차이팟 에어팟 TWS i7 i9 i12 블루투스 무선 이어폰,19562859308,0
7,요이치 에어 포스 팟 무선 충전 2세대,20991114077,148
8,애플 에어팟 2세대 유선충전 케이스,18344345265,1201
9,당일발송 애플 에어팟 2세대 한쪽 왼쪽 오른쪽 유닛 본체 충전케이스 애플코리아정품,82180951465,201


In [21]:
df.to_csv("네이버쇼핑검색-에어팟.csv")
# 셀 안의 내용이 너무 길면 다음과 같은 에러가 발생한다. img_url, url이 이에 해당하기 때문에 df에서 이 컬럼을 삭제한다.


In [269]:
df.drop(['goods_url', 'goods_img_url'], axis=1, inplace=True)
df.head()

,goods_title,goods_price,goods_cat,spec,review_count,star_point,created_at,nvmid,is_ad,chpid,keep_count
0,애플 에어팟 2세대 유선충전 (MV7N2KH/A),89500,"[디지털/가전, 음향가전, 블루투스셋, 블루투스이어폰/이어셋]","{'종류': '이어셋', '무선방식': '블루투스(애플H1칩탑재)', '통화': '...",11131,None,2019.04.,18622086330,False,,4488
1,애플 에어팟 1세대 (MMEF2KH/A),144900,"[디지털/가전, 음향가전, 블루투스셋, 블루투스이어폰/이어셋]","{'채널': '스테레오', '무선방식': '블루투스(애플W1칩탑재)', '구성품':...",11591,None,2016.12.,10776906666,False,,3168
2,애플 에어팟 2세대 무선충전 (MRXJ2KH/A),83000,"[디지털/가전, 음향가전, 블루투스셋, 블루투스이어폰/이어셋]","{'무선방식': '블루투스(애플H1칩탑재)', '통화': '3시간', '구성품': ...",3636,None,2019.04.,18754856889,False,,2576
3,애플 에어팟 왼쪽 오른쪽 단품 한쪽 1세대 2세대 당일발송 국내정식발매품 에어팟2 2세대,67000,"[디지털/가전, 음향가전, 블루투스셋, 블루투스이어폰/이어셋]","{'종류': '이어셋', '채널': '스테레오', '무선방식': '블루투스(애플W1...",782,None,2019.03.,81946053258,False,4401530808,642
4,[MLC] 애플 에어팟 2세대 유선/무선충전,179450,"[디지털/가전, 음향가전, 블루투스셋, 블루투스이어폰/이어셋]",{},496,None,2019.06.,19739090251,False,57360784,0


In [270]:
df.to_excel("네이버쇼핑검색-에어팟.xlsx")